In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

import wrangle as w

In [2]:
# acquire data
w.acquire_df()
w.prepare_df()

,cohort_id,cohort_name,start_date,end_date,program_id,dates,time,endpoint,user_id,ip
0,8.0,Hampton,2015-09-22,2016-02-06,1.0,2018-01-26,09:55:03,/,1,97.105.19.61
1,8.0,Hampton,2015-09-22,2016-02-06,1.0,2018-01-26,09:56:02,java-ii,1,97.105.19.61
2,8.0,Hampton,2015-09-22,2016-02-06,1.0,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,97.105.19.61
3,8.0,Hampton,2015-09-22,2016-02-06,1.0,2018-01-26,09:56:06,slides/object_oriented_programming,1,97.105.19.61
4,22.0,Teddy,2018-01-08,2018-05-17,2.0,2018-01-26,09:56:24,javascript-i/conditionals,2,97.105.19.61
...,...,...,...,...,...,...,...,...,...,...
900217,28.0,Staff,2014-02-04,2014-02-04,2.0,2021-04-21,16:41:51,jquery/personal-site,64,71.150.217.33
900218,28.0,Staff,2014-02-04,2014-02-04,2.0,2021-04-21,16:42:02,jquery/mapbox-api,64,71.150.217.33
900219,28.0,Staff,2014-02-04,2014-02-04,2.0,2021-04-21,16:42:09,jquery/ajax/weather-map,64,71.150.217.33
900220,28.0,Staff,2014-02-04,2014-02-04,2.0,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,24.160.137.86


In [3]:
# preview data
df.head()

NameError: name 'df' is not defined

In [ ]:
# df info
df.info()

In [ ]:
# df describe 
df.describe()

In [ ]:
# view nulls
df.isnull().sum()

# 1 null in endpoint and 52893 in cohort

In [ ]:
# view endpoint null 2020-04-08 09:25:18 user: 586 cohort: 55.0 ip: 72.177.240.51
df[df.endpoint.isnull()]

In [ ]:
# view cohort nulls 78 unique users with cohort nulls -- possibly instructors or admin
cohort_nulls = df[df.cohort.isnull()]
user_cohort_null_list = cohort_nulls.user_id.unique()

In [ ]:
# checks to see if user id has nothing but nans for cohort and places into nan_only list or nan_and_other
nan_only = []
nan_and_other = []

for u in user_cohort_null_list:
    null_cohort = df[df.user_id == u].cohort.unique()
    
    if np.isnan(null_cohort).all():
        nan_only.append(u)
    else:
        nan_and_other.append(u)


In [ ]:
# there are 8 
len(nan_and_other)
nan_and_other

In [ ]:
# check ids and cohorts
for u in nan_and_other:
    print(u)
    print(df[df.user_id == u].cohort.unique())
    
# 358, 375, 644, 744 only had one cohort
# all other ids have multiple cohorts

In [ ]:
# there are 70 ids with no cohort attached
len(nan_only)

**Question**
> If someone graduates will they always have that same cohort attached to them even after graduation?

> how many cohorts are there?

> how many people are in each cohort?


In [ ]:
# how many cohorts are there?
# 47 cohorts minus the nan

# list of cohorts
cohort_list = list(df.cohort.unique())

In [ ]:
# cohort names ids
cohort_df = pd.read_csv("curriculum_logs_cohorts.csv")

In [ ]:
# how many people are in each cohort?
# number cohort and number of users
for n in cohort_list:
    print(f"Cohort: {n}")
    users_of_cohort = df[df.cohort == n].user_id.unique()
    print(f"Number of Users: {len(users_of_cohort)}")


In [ ]:
# user_ids in each cohort
cohort_dict = {}

for n in cohort_list:
    users_of_cohort = df[df.cohort == n].user_id.unique()
    users_of_cohort = list(users_of_cohort) 
    cohort_dict[n] = users_of_cohort

cohort_dict

In [ ]:
# change cohort nulls to 0
df.cohort = df.cohort.fillna(0)

In [ ]:
# drop 1 null in endpoint
df = df.dropna()

In [ ]:
# Remove '\t\t' character from col1
df.dates = df.dates.str.replace("\t\t", "")
df

In [ ]:
# check nulls
df.isnull().sum()

In [ ]:
pd.set_option('display.max_colwidth',1000)
# finding unique pages -- 2313 pages
df.endpoint.value_counts()

In [ ]:
js_result = df[df['endpoint'].str.startswith('javascript')]

In [ ]:
js_result.columns


In [ ]:
# rename cohort to id to match 
temp_df = df.copy()

In [ ]:
# cohort names ids
cohort_df = pd.read_csv("curriculum_logs_cohorts.csv")

In [ ]:
cohort_df.columns

In [ ]:
cohort_df = cohort_df.rename(columns = {"id": "cohort"})

In [ ]:
cohort_df

In [ ]:
temp_df

In [ ]:
df = pd.merge(cohort_df, temp_df,  on=['cohort'], how='inner')

In [ ]:
df

In [ ]:
# rename columns 'cohort', 'cohort_name', 'time', 'endpoint', 'user_id', 'cohort', 'ip' 'start_date', 'end_date', 'program_id'
df = df.drop(columns=[ 'slack',  'created_at', 'updated_at', 'deleted_at'])

In [ ]:
df.info()

In [ ]:
df.start_date = df.start_date.astype("datetime64")
df.end_date = df.end_date.astype("datetime64")

In [ ]:
# update dtype for dates to datetime64
df.dates = df.dates.astype("datetime64")

In [ ]:
df.info()

In [ ]:
def prepare_df():
    # read df
    df = w.acquire_df()
    # ----handle nulls
    # change cohort nulls to 0
    df.cohort = df.cohort.fillna(0)
    
    

    # drop 1 null in endpoint
    df = df.dropna()

    # Remove '\t\t' character from col1
    df.dates = df.dates.str.replace("\t\t", "")
    

    # read cohort df
    cohort_df = pd.read_csv("curriculum_logs_cohorts.csv")
    
    # ----- merge tables
    # rename id to cohort to match
    cohort_df = cohort_df.rename(columns = {"id": "cohort"})

    # merge on cohort
    df = pd.merge(cohort_df, df,  on=['cohort'], how='right')

    # drop columns
    df = df.drop(columns=[ 'slack',  'created_at', 'updated_at', 'deleted_at'])

    # update dtype for dates to datetime64
    df.start_date = df.start_date.astype("datetime64")
    df.end_date = df.end_date.astype("datetime64")
    df.dates = df.dates.astype("datetime64")

    return df  


In [ ]:
prepare_df()

In [ ]:
w.prepare_df()